In [ ]:
from zipfile import ZipFile

file_name = 'compressed.zip'

with ZipFile(file_name, 'r') as f:
  f.extractall()
  print("Done")

Done


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import sklearn.metrics as metrics
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.applications import VGG16
import cv2
import imghdr
import os
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
rm train/.DS_Store

rm: cannot remove 'train/.DS_Store': No such file or directory


In [ ]:
rm test/.DS_Store

rm: cannot remove 'test/.DS_Store': No such file or directory


In [ ]:
folders = ['train', 'test']
image_exts = ['jpeg','jpg', 'bmp', 'png']

for folder in folders:
  for image_class in os.listdir(folder): 
      print(image_class)
      for image in os.listdir(os.path.join(folder, image_class)):
          image_path = os.path.join(folder, image_class, image)
          try: 
              img = cv2.imread(image_path)
              tip = imghdr.what(image_path)
              if tip not in image_exts: 
                  print('Image not in ext list {}'.format(image_path))
                  os.remove(image_path)
          except Exception as e: 
              print('Issue with image {}'.format(image_path))
              # os.remove(image_path)

white_knight
black_knight
empty
white_pawn
black_queen
white_queen
white_king
black_pawn
white_rook
black_bishop
white_bishop
black_king
black_rook
white_knight
black_knight
empty
white_pawn
black_queen
white_queen
white_king
black_pawn
white_rook
black_bishop
white_bishop
black_king
black_rook


In [ ]:
trdata = ImageDataGenerator(validation_split=0.3)
train_data_gen = trdata.flow_from_directory(directory="train", target_size=(256,256), shuffle=True, class_mode="categorical", subset="training")
validation_data_gen = trdata.flow_from_directory(directory="train", target_size=(256,256), shuffle=True, class_mode="categorical", subset="validation")
tsdata = ImageDataGenerator()
test_data_gen = tsdata.flow_from_directory(directory="test", target_size=(256,256),shuffle=True, class_mode="categorical")

Found 1665 images belonging to 13 classes.
Found 703 images belonging to 13 classes.
Found 832 images belonging to 13 classes.


In [ ]:
vgg = VGG16(input_shape = (256,256,3), weights = "imagenet", include_top = False)
for layer in vgg.layers:
 layer.trainable = False
x = Flatten()(vgg.output)
x = Dense(128, activation = "relu", kernel_initializer="he_normal")(x) 
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(rate=0.5)(x)
x = Dense(64, activation = "relu", kernel_initializer="he_normal")(x) 
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(rate=0.5)(x)
x = Dense(13, activation = "softmax")(x) 
model = Model(inputs = vgg.input, outputs = x)

model.compile(loss="categorical_crossentropy", optimizer="RMSprop", metrics=["accuracy"])

In [ ]:
training_steps_per_epoch = np.ceil(train_data_gen.samples / 20)
validation_steps_per_epoch = np.ceil(validation_data_gen.samples / 20)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True,verbose=1)

model.fit_generator(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=validation_data_gen, validation_steps=validation_steps_per_epoch,epochs=10, verbose=1, callbacks=[early_stopping_cb])

Epoch 1/10


<ipython-input-15-8ad1c482765c>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=validation_data_gen, validation_steps=validation_steps_per_epoch,epochs=10, verbose=1, callbacks=[early_stopping_cb])


53/84 [=================>............] - ETA: 11:06 - loss: 1.9859 - accuracy: 0.4180

84/84 [==============================] - 1646s 20s/step - loss: 1.9859 - accuracy: 0.4180 - val_loss: 3.1328 - val_accuracy: 0.2632
